In [161]:
import pandas as pd
import glob
import numpy as np
import math

In [16]:
train_set_dir = "/media/data2/Data/wsdm2019/data/training_set/"
large_train_set_dir = "/media/data2/Data/wsdm2019/python/data/training_set/"
test_set_dir = "/media/data2/Data/wsdm2019/data/test_set/"
track_features_dir = "/media/data2/Data/wsdm2019/python/data/track_features/"

output_dir = "/media/data2/Data/wsdm2019/python/data/train_examples/"

In [17]:
train_files = sorted(glob.glob(train_set_dir+"*.csv"))

In [20]:
num_combine= 66
files_chunk = []
for i in range(0, len(train_files), num_combine):
    files_chunk.append(train_files[i: i+ num_combine])

len(files_chunk)
files_chunk[1]

['/media/data2/Data/wsdm2019/data/training_set/log_1_20180715_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180716_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180717_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180718_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180719_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180720_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180721_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180722_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180723_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180724_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180725_000000000000.csv',
 '/media/data2/Data/wsdm2019/data/training_set/log_1_20180726_000000000000.csv',
 '/media/data2/Data/wsdm2019

In [21]:
test_file = train_files[0]

In [22]:
df = pd.read_csv(test_file)

In [297]:
df.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,t_64f3743c-f624-46bb-a579-0f3f9a07a123,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone


In [15]:
df[['skip_1','skip_2','skip_3','not_skipped']].drop_duplicates()

,skip_1,skip_2,skip_3,not_skipped
0,False,False,False,True
5,False,False,True,False
6,True,True,True,False
8,False,True,True,False
22,False,False,False,False
60932,False,True,False,False


In [300]:
def convert_feature_to_map(feature_list):
    # insert to the 0th position the PAD string as padding
#     print(feature_list.shape)
#     feature_map = {'PAD': 0}
#     feature_map.update(dict(zip(feature_list, range(1, 1+ len(feature_list)))))
    feature_map = dict(zipo(feature_list, range(len(feature_list))))
    
    print(feature_map)
    return feature_map, len(feature_list)

In [393]:
bool_columns = ['skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch', 'no_pause_before_play', 'short_pause_before_play', 
                'long_pause_before_play', 'hist_user_behavior_is_shuffle', 'premium']
numerical_columns = ['hist_user_behavior_n_seekfwd', 'hist_user_behavior_n_seekback']
numerical_columns_range_map = {'hist_user_behavior_n_seekfwd': (0,100), 'hist_user_behavior_n_seekback': (0,100)}
dropped_columns = ['session_position', 'session_length', 'hour_of_day', 'date'] # temporaryly dropped, will add back if useful
categorical_columns = ['context_type', 'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end']
categorical_columns_map = {'context_type': 
                           {'catalog': 0, 'charts': 1, 'editorial_playlist': 2, 'personalized_playlist': 3, 'radio': 4, 'user_collection': 5},
                           'hist_user_behavior_reason_start':
                           {'appload': 0, 'backbtn': 1, 'clickrow': 2, 'endplay': 3, 'fwdbtn': 4, 'playbtn': 5, 'popup': 6, 'remote': 7, 'trackdone': 8, 'trackerror': 9, 'uriopen': 10, 'clickside': 11},
                            'hist_user_behavior_reason_end':
                           {'appload': 0, 'backbtn': 1, 'clickrow': 2, 'endplay': 3, 'fwdbtn': 4, 'logout': 5, 'popup': 6, 'remote': 7, 'trackdone': 8, 'uriopen': 9, 'clickside': 10}
}
id_column = ['session_id', 'track_id_clean']

In [394]:
len(categorical_columns_map['hist_user_behavior_reason_end'])

11

In [395]:
df_features = df.drop(['session_id', 'track_id_clean'], axis=1)
new_df = df.copy(deep=True)
for feature in df_features.columns:
    if feature in bool_columns:
        # for all the bool features, convert to integer [0 --> False, 1--> True]
        new_df[feature] = new_df[feature].astype(int) 
    elif feature in numerical_columns:
        # for numerical columns, define min and max for processing
        # do min-max scaling which is (x - min) / (max - min), if value is bigger than max set to max
        (min, max) = numerical_columns_range_map[feature]
        new_df[feature] = new_df[feature].clip(min, max)
        new_df[feature] = (new_df[feature] - min) / (max - min)
    elif feature in categorical_columns:
        # if feature is categorical, convert feature into a map of item --> integer, and leave integer 0 for padding as bool
#         feature_map, feature_length = convert_feature_to_map(new_df[feature].drop_duplicates().sort_values().values)
        new_df[feature] = new_df[feature].map(categorical_columns_map[feature])
    elif feature in dropped_columns:
        new_df = new_df.drop(feature, axis=1)

# sanity check
print("if there is nan: " + str(new_df.isnull().values.any()))

if there is nan: False


In [396]:
new_df.columns

Index(['session_id', 'track_id_clean', 'skip_1', 'skip_2', 'skip_3',
       'not_skipped', 'context_switch', 'no_pause_before_play',
       'short_pause_before_play', 'long_pause_before_play',
       'hist_user_behavior_n_seekfwd', 'hist_user_behavior_n_seekback',
       'hist_user_behavior_is_shuffle', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end'],
      dtype='object')

In [397]:
df_group = new_df.groupby('session_id')

In [398]:
test_group = df_group.get_group('0_00006f66-33e5-4de7-a324-2d18e439fc1e')

In [399]:
test_group

,session_id,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,0,0,0,1,0,0,0,0,0.0,0.0,1,1,2,8,8
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_64f3743c-f624-46bb-a579-0f3f9a07a123,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
5,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_c815228b-3212-4f9e-9d4f-9cb19b248184,0,0,1,0,0,1,0,0,0.0,0.0,1,1,2,8,4
6,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_e23c19f5-4c32-4557-aa44-81372c2e3705,1,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4
7,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0be6eced-f56f-48bd-8086-f2e0b760fdee,1,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4
8,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_f3ecbd3b-9e8e-4557-b8e0-39cfcd7e65dd,0,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4
9,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_2af4dfa0-7df3-4b7e-b7ab-353ba48237f9,1,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4


In [400]:
sequence_length = len(test_group)
cut_value = math.floor(sequence_length / 2)

In [401]:
test_group_history = test_group.iloc[:cut_value]
test_group_predict = test_group.iloc[cut_value:][['session_id', 'track_id_clean']]

In [402]:
test_group.iloc[cut_value:]['skip_2']

10    1
11    1
12    1
13    0
14    1
15    1
16    1
17    1
18    1
19    1
Name: skip_2, dtype: int64

In [403]:
test_group_history

,session_id,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,0,0,0,1,0,0,0,0,0.0,0.0,1,1,2,8,8
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_64f3743c-f624-46bb-a579-0f3f9a07a123,0,0,0,1,0,1,0,0,0.0,0.0,1,1,2,8,8
5,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_c815228b-3212-4f9e-9d4f-9cb19b248184,0,0,1,0,0,1,0,0,0.0,0.0,1,1,2,8,4
6,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_e23c19f5-4c32-4557-aa44-81372c2e3705,1,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4
7,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0be6eced-f56f-48bd-8086-f2e0b760fdee,1,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4
8,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_f3ecbd3b-9e8e-4557-b8e0-39cfcd7e65dd,0,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4
9,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_2af4dfa0-7df3-4b7e-b7ab-353ba48237f9,1,1,1,0,0,1,0,0,0.0,0.0,1,1,2,4,4


In [404]:
test_group_predict

,session_id,track_id_clean
10,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_79570b2a-639a-4ec0-9853-71c5299ac44d
11,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_69016f19-84aa-40c0-afa9-54404397b7a2
12,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_59dc3fcd-7aec-4da5-a747-b59b19bab3bb
13,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_a157ba98-7eae-4e7d-99d5-ed760b2c0978
14,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_87d95b75-af5c-4ef6-8dc4-cd888ae17cce
15,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_db945033-3dc5-4a22-8889-c72fd2781299
16,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_bff5b6c6-6968-41be-b723-7e860b372975
17,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_bf496d77-9eb2-4fe1-80f7-2c7ecd6f9b8e
18,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_1051bc37-1a73-4301-812b-f83ae0c9bbcd
19,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_358c9cce-7a1e-4dd4-81de-206dda80363f


In [405]:
import pickle
with open (track_features_dir + "track2idx_all.pkl", "rb") as f:
    track2idx = pickle.load(f)

In [406]:
b = [track2idx[a] for a in test_group_history['track_id_clean'].values]

In [407]:
b

[2219009,
 1275968,
 1701710,
 2222640,
 3236110,
 386889,
 2899402,
 373655,
 934671,
 2633090]

In [408]:
c = [track2idx[a] for a in test_group_predict['track_id_clean']]

In [409]:
c

[2635577,
 2243512,
 582731,
 2062828,
 2238620,
 2634102,
 2904306,
 933915,
 2068051,
 641486]

In [410]:
test_group_history[bool_columns].values

array([[0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1]])

In [411]:
test_group_history[numerical_columns].values

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [412]:
test_group_history[categorical_columns].values

array([[2, 8, 8],
       [2, 8, 8],
       [2, 8, 8],
       [2, 8, 8],
       [2, 8, 8],
       [2, 8, 4],
       [2, 4, 4],
       [2, 4, 4],
       [2, 4, 4],
       [2, 4, 4]])

In [413]:
# getting all the maps (track map and skip2 map)

import pickle
with open (track_features_dir + "track2idx_all.pkl", "rb") as f:
    track2idx = pickle.load(f)

In [414]:
ngroups = df_group.ngroups
print(ngroups)

178342


In [415]:
examples = []
counter = 0
for name, group in df_group:
    sequence_length = len(group)
    cut_value = math.floor(sequence_length / 2)
    
    group_history = group.iloc[:cut_value]
    group_predict = group.iloc[cut_value:]
    
    # group target is the group skip_2 value [0 --> False, 1 --> True]
    group_target = group_predict['skip_2'].values
    
    # extract tracks data
    tracks_history = [track2idx[a] for a in group_history['track_id_clean'].values]
    tracks_predict = [track2idx[a] for a in group_predict['track_id_clean'].values]
    
    # extract bool data
    tracks_feature_bool = group_history[bool_columns].values
    
    # extract numerical data
    tracks_feature_numerical = group_history[numerical_columns].values
    
    # extract categorical data
    tracks_feature_categorical = group_history[categorical_columns].values
    
    examples.append({
        'tracks_history': tracks_history,
        'tracks_predict': tracks_predict,
        'tracks_feature_bool': tracks_feature_bool,
        'tracks_feature_numerical': tracks_feature_numerical,
        'tracks_feature_categorical': tracks_feature_categorical,
        'target': group_target
    })
    
    counter += 1
    if (counter % 10000 == 0):
        print(counter / ngroups)

# sanity check
print(len(examples))

0.05607204135873771
0.11214408271747542
0.1682161240762131
0.22428816543495084
0.2803602067936885
0.3364322481524262
0.39250428951116395
0.4485763308699017
0.5046483722286393
0.560720413587377
0.6167924549461148
0.6728644963048525
0.7289365376635902
0.7850085790223279
0.8410806203810656
0.8971526617398033
0.953224703098541
178342


In [416]:
import pickle
with open(output_dir + "sample_examples", "wb") as f:
    pickle.dump(examples, f)

In [417]:
print(examples[0])

{'tracks_history': [2219009, 1275968, 1701710, 2222640, 3236110, 386889, 2899402, 373655, 934671, 2633090], 'tracks_predict': [2635577, 2243512, 582731, 2062828, 2238620, 2634102, 2904306, 933915, 2068051, 641486], 'tracks_feature_bool': array([[0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1]]), 'tracks_feature_numerical': array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]), 'tracks_feature_categorical': array([[2, 8, 8],
       [2, 8, 8],
       [2, 8, 8],
       [2, 8, 8],
       [2, 8, 8],
       [2, 8, 4],
       [2, 4, 4],
       [2, 4, 